# Key Terms: 
Calculate the TF-IDF(Term Frequency-Inverse Document Frequency) for a given subreddit.

Produce a Tag Cloud of the terms (note: this doesn’t have to be integrated into your code; simply including the image is enough).

In [1]:
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.sql.types import StringType, DoubleType
import pyspark.sql.functions as func
import re
import math

def pre_process(text):
    # lowercase
    text=text.lower()
    
    # remove special characters and digits
    text=re.sub("(\\d|[^\\w|\\s]|(\_))+","",text)
    text=re.sub("(\\s)+"," ",text)
    print(text)
    return text.strip()

def calc_idf(docCount, df):
    return math.log((float(docCount) + 1) / (float(df) + 1))
#     return math.log((float(docCount) + 1))

pre_process_udf = func.udf(pre_process, StringType())
calc_idf_udf = func.udf(calc_idf, DoubleType())

Import data from sampled_reddit dataset (10% of whole dataset)

In [2]:
df = sqlContext.read.json("hdfs://orion11:11001/sampled_reddit/*")
# df = sqlContext.read.json("hdfs://orion11:11001/reddit/2006/*")

Get the list of subreddit with the number of comment in their. Then sort and get the subreddit that have the most comment

In [3]:
subredditComments = (df
 .groupBy(df.subreddit)
 .agg(
     func.count(func.lit(1)).alias("Num Of Comments")
     ))
(subredditComments
 .sort(func.desc("Num Of Comments"))
 .show())

+---------------+---------------+
|      subreddit|Num Of Comments|
+---------------+---------------+
|      AskReddit|       28466878|
|          funny|        6385225|
|           pics|        6015627|
|       politics|        5154982|
|leagueoflegends|        4811568|
|         gaming|        4165456|
|            WTF|        3583591|
|      worldnews|        3456812|
|  AdviceAnimals|        3408410|
|         videos|        3335506|
|            nfl|        3251565|
|  todayilearned|        2922639|
|            nba|        2679799|
|           news|        2411619|
|         soccer|        2222209|
|           IAmA|        2145773|
|         movies|        1978592|
|          DotA2|        1880917|
|        atheism|        1878108|
|   pcmasterrace|        1798732|
+---------------+---------------+
only showing top 20 rows



Base on the result, we will select AskReddit as the one to calculate TF-IDF
Then we only select the body(comment) and generate id for each comment

In [4]:
askRedditComments = (df
                     .filter(df.subreddit.like("AskReddit") & ~df.body.like('[removed]') & ~df.body.like('[deleted]'))
                     .select(df.body))
# askRedditComments = df.filter(df.subreddit.like("reddit.com")).select(df.body)
# askRedditComments = (df
#                      .filter(df.subreddit.like("slate") & ~df.body.like('[removed]') & ~df.body.like('[deleted]'))
#                      .select(df.body))

askRedditComments = (askRedditComments
                     .withColumn("body", pre_process_udf(askRedditComments.body))
                     .withColumn("doc_id", func.monotonically_increasing_id()))
docCount = askRedditComments.count()
askRedditComments.show()
askRedditComments.count()

+--------------------+------+
|                body|doc_id|
+--------------------+------+
|i read the title ...|     0|
|because youre abo...|     1|
|flushing with you...|     2|
|              me too|     3|
|i dont know but m...|     4|
|we make a differe...|     5|
|isnt summer heigh...|     6|
|i got mine out a ...|     7|
|or show us you ex...|     8|
|defiantly im not ...|     9|
|pretty sure once ...|    10|
|while this does s...|    11|
|as long as they c...|    12|
|i use formatfacto...|    13|
|heman sings it be...|    14|
|                 see|    15|
|dont be shy cat s...|    16|
|for the time it w...|    17|
|i know i will nev...|    18|
|yeah right story ...|    19|
+--------------------+------+
only showing top 20 rows



26264703

In [5]:
commentsTokensDF = (askRedditComments
                    .select("body", "doc_id",func.explode(func.split("body", "\s+")).alias("word")))
commentsTokensDF.show()

+--------------------+------+--------+
|                body|doc_id|    word|
+--------------------+------+--------+
|i read the title ...|     0|       i|
|i read the title ...|     0|    read|
|i read the title ...|     0|     the|
|i read the title ...|     0|   title|
|i read the title ...|     0|     and|
|i read the title ...|     0| thought|
|i read the title ...|     0|      of|
|i read the title ...|     0|    that|
|i read the title ...|     0|cheating|
|i read the title ...|     0|   bitch|
|i read the title ...|     0|   clown|
|i read the title ...|     0|    from|
|i read the title ...|     0|     the|
|i read the title ...|     0|glassjaw|
|i read the title ...|     0|   video|
|because youre abo...|     1| because|
|because youre abo...|     1|   youre|
|because youre abo...|     1|   about|
|because youre abo...|     1|      to|
|because youre abo...|     1|    wash|
+--------------------+------+--------+
only showing top 20 rows



Calculate the TF(Term Frenquency) - the number of time that work appear in each comments

In [6]:
wordTf = (commentsTokensDF.groupBy("doc_id","word")
        .agg(func.count("body").alias("tf")))
wordTf.show()

+------+--------------------+---+
|doc_id|                word| tf|
+------+--------------------+---+
|     4|              showed|  1|
|     7|               thing|  2|
|    21|                  do|  1|
|    27|                 why|  1|
|    42|                  as|  1|
|    46|                 its|  1|
|    50|                turn|  1|
|    51|               start|  1|
|    55|                 and|  2|
|    91|                 gym|  1|
|   129|thesehttptoiletcl...|  1|
|   142|                time|  2|
|   175|                fuck|  1|
|   176|                  no|  1|
|   183|               youre|  1|
|   194|                kind|  1|
|   207|                 the|  1|
|   235|                sure|  1|
|   240|                 age|  1|
|   258|               under|  1|
+------+--------------------+---+
only showing top 20 rows



Calculate the DF(Document Frequency) - the number of comments that contain that word

In [7]:
wordDf = (commentsTokensDF.groupBy("word")
        .agg(func.countDistinct("doc_id").alias("df")))
wordDf.show()

+---------------+-----+
|           word|   df|
+---------------+-----+
|          aaagh|   34|
|         abbrev|   11|
|   accumulation|  567|
|        acidity|  515|
|         aholes|  356|
|      airheaded|   77|
|      amplifier|  407|
|          anime|17660|
|       antennae|  254|
|          apgar|   31|
|      arguments|18758|
|            art|47713|
|          aruba|  218|
|atheistsatheist|    1|
|       autisitc|    3|
|       backdate|   19|
|        balding| 1749|
|        baloons|   76|
|        barrier| 5336|
|      barristan|   91|
+---------------+-----+
only showing top 20 rows



Base on DF, we will calculate IDF(Inverse Document Frequency)   
IDF(t,D) = log[ (|D| + 1) / (DF(t,D) + 1) ]

In [8]:
wordIdf = (wordDf
           .withColumn("idf", calc_idf_udf(func.lit(docCount), wordDf.df)))
wordIdf.show()

+----------------+-----+------------------+
|            word|   df|               idf|
+----------------+-----+------------------+
|    accumulation|  567|10.741615123834924|
|         acidity|  515|10.837629777074513|
|       advisoras|    1| 16.39058936199613|
|          aholes|  356|11.206000760776437|
|       amplifier|  407|11.072469368151914|
|       anathesia|    4|15.474298630121975|
|           anime|17660| 7.304622444859705|
|           anodd|    5|15.291977073328022|
|        antennae|  254| 11.54247299739765|
|        apagaron|    1| 16.39058936199613|
|      applejuice|   40|13.370164475851768|
|       arguments|18758| 7.244307626320668|
|     arrivedleft|    1| 16.39058936199613|
|        arsholes|    3|15.697442181436186|
|             art|47713|6.3107564076944325|
|           aruba|  218|11.694664812739575|
|asimovsilverberg|    2|15.985124253887966|
|             avx|    8|14.886511965219857|
|         ayyyyyy|  113| 12.34753809416158|
|           baaaa|   62|12.94060

Calculate TF-IDF base on wordTf and wordIdf above by multiply it  
Which will generate a matrix of each word for document

In [10]:
wordTfIdf = (wordTf
      .join(wordIdf, ["word"])
      .withColumn("tf_idf", wordTf.tf * wordIdf.idf))

In [14]:
wordTfIdf.filter(wordTfIdf.word.like("two")).sort(func.desc("tf_idf")).show()

+----+--------------+---+------+---------------+----------------+
|word|        doc_id| tf|    df|            idf|          tf_idf|
+----+--------------+---+------+---------------+----------------+
| two| 2018634641920| 21|527696|3.9074590082268|82.0566391727628|
| two| 6012954220978| 21|527696|3.9074590082268|82.0566391727628|
| two| 3496103386112| 16|527696|3.9074590082268|62.5193441316288|
| two| 8521215117327| 16|527696|3.9074590082268|62.5193441316288|
| two| 6356551598664| 15|527696|3.9074590082268| 58.611885123402|
| two|16733192592064| 13|527696|3.9074590082268|50.7969671069484|
| two|  884763267346| 12|527696|3.9074590082268|46.8895080987216|
| two| 2516850844485| 12|527696|3.9074590082268|46.8895080987216|
| two|16003048152114| 12|527696|3.9074590082268|46.8895080987216|
| two| 1133871386678| 12|527696|3.9074590082268|46.8895080987216|
| two| 2851858285444| 12|527696|3.9074590082268|46.8895080987216|
| two|13838384632652| 12|527696|3.9074590082268|46.8895080987216|
| two|1157

### Tag clouds
Which terms should i put into tag cloud and how the value will be determined